In [179]:
from langchain.llms import OpenAI

In [180]:
import os

In [181]:
# llm = OpenAI(temperature=0.6)
# temperature: 0.0 - 1.0 (when increased, the model becomes more creative and less factual)

In [182]:
text = 'what is the capital of France?'
# print(llm.predict(text))

In [183]:
os.environ['HUGGINGFACEHUB_API_TOKEN']=""
os.environ['OPENAI_API_KEY']=''

In [184]:
from langchain import HuggingFaceHub
llm_huggingface= HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":0,"max_length":64 },huggingfacehub_api_token= os.getenv("HUGGINGFACEHUB_API_TOKEN"))

In [185]:
output=llm_huggingface.predict("can you write a poem about AI")
print(output)

i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love you i love


In [186]:
# llm.predict("can you write a poem about AI")

### Prompt Templates and LLM chains

In [187]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=['country'],template="Tell me the capital of {country}")

prompt_template.format(country="India")

'Tell me the capital of India'

In [188]:
from langchain.chains import LLMChain
chain = LLMChain(llm=llm_huggingface , prompt= prompt_template)
chain.run("India")

'calcutta'

### Combining multiple chains using simple sequential chain

In [189]:
capital_template = PromptTemplate(input_variables=['country'],template="Tell me the capital of {country}")

capital_chain=LLMChain(llm=llm_huggingface , prompt= capital_template)

famous_template =PromptTemplate(input_variables=['capital'], template="suggest me some famous places in {capital}")

In [190]:
famous_chain=LLMChain(llm=llm_huggingface, prompt=famous_template)

In [191]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("India")

'The Museum of Modern Art'

### Sequential chain

In [192]:
capital_template = PromptTemplate(input_variables=['country'],template="Tell me the capital of {country}")

capital_chain=LLMChain(llm=llm_huggingface , prompt= capital_template, output_key="capital")

In [193]:
famous_template =PromptTemplate(input_variables=['capital'], template="suggest me some famous places in {capital}")

famous_chain=LLMChain(llm=llm_huggingface, prompt=famous_template, output_key="places")

In [194]:
from langchain.chains import SequentialChain
chain= SequentialChain(chains=[capital_chain,famous_chain], input_variables=['country'],output_variables=['capital','places'])

In [195]:
chain({'country':"india"})

{'country': 'india',
 'capital': 'chennai',
 'places': 'The Temple of Lord Vishnu'}

### Chatmodels with ChatOpenAI

In [196]:
from langchain.chat_models import ChatOpenAI

In [197]:
from langchain.schema import HumanMessage, SystemMessage ,AIMessage

In [198]:
chatllm=ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],temperature=0.6,model='gpt-3.5-turbo')

In [199]:
chatllm([
    SystemMessage(content="You are a comedian AI assistant"),
    HumanMessage(content="Please provide some comedy punchlines on AI"),
])

AIMessage(content='1. "Why did the AI cross the road? To optimize its route and avoid traffic congestion!"\n\n2. "Why did the AI go to therapy? It had a case of binary codependency!"\n\n3. "Why did the AI get kicked out of the comedy club? It kept confusing punchlines with punch cards!"\n\n4. "Why did the AI become a stand-up comedian? It wanted to compute laughter algorithms and crack some jokes!"\n\n5. "Why did the AI start a band? It wanted to create artificial harmony and make some byte-sized tunes!"\n\n6. "Why did the AI take up gardening? It wanted to learn how to cultivate artificial intelligence... and some pretty flowers!"\n\n7. "Why did the AI become a chef? It wanted to cook up some byte-sized bites and serve up a taste of the future!"\n\n8. "Why did the AI join a dance class? It wanted to learn some new moves... and maybe do the robot!"\n\n9. "Why did the AI start a fashion line? It wanted to design cutting-edge algorithms and make artificial intelligence look fabulous!"\n\

### Prompt Template + LLM + Output Parsers

In [200]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [201]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [202]:
template="You are a helpful assistant. When the user gives any input, you should generate a 5 words synonyms comma seperated list "
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [203]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [204]:
chain.invoke({"text":"intelligent"})

['smart', ' clever', ' bright', ' sharp', ' quick-witted']